In [ ]:
print('train data shape :', train.shape)
print('test data shape :', test.shape)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/widsdatathon2023/train_data.csv
/kaggle/input/widsdatathon2023/test_data.csv
/kaggle/input/widsdatathon2023/sample_solution.csv


  ## IMPORTING LIBRARIES

In [5]:
# dataframe and plotting
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# machine learning
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

## LOAD DATASETS FROM KAGGLE

In [3]:
train = pd.read_csv('/kaggle/input/widsdatathon2023/train_data.csv')
test = pd.read_csv('/kaggle/input/widsdatathon2023/test_data.csv')
ss = pd.read_csv('/kaggle/input/widsdatathon2023/sample_solution.csv')

In [4]:
#reduce the size of the memory usage by the training set
from fastai.tabular.core import df_shrink
train_shrink = df_shrink(train)
print(train_shrink.memory_usage().sum()/2**20)
train = train_shrink

347.60015869140625


## EDA

In [33]:
ss.head()

,contest-tmp2m-14d__tmp2m,index
0,27.073876,375734
1,25.109308,375735
2,22.557390,375736
3,25.572875,375737
4,20.781073,375738


In [ ]:
pd.set_option('display.max_columns', None) #to display all the columns
train.head()


In [13]:
# Check for missing values
pd.set_option('display.max_rows', None) #display all the rows
print('missing values:', train.isnull().sum())

missing values: index                                           0
lat                                             0
lon                                             0
startdate                                       0
contest-pevpr-sfc-gauss-14d__pevpr              0
nmme0-tmp2m-34w__cancm30                        0
nmme0-tmp2m-34w__cancm40                        0
nmme0-tmp2m-34w__ccsm30                     15934
nmme0-tmp2m-34w__ccsm40                         0
nmme0-tmp2m-34w__cfsv20                         0
nmme0-tmp2m-34w__gfdlflora0                     0
nmme0-tmp2m-34w__gfdlflorb0                     0
nmme0-tmp2m-34w__gfdl0                          0
nmme0-tmp2m-34w__nasa0                          0
nmme0-tmp2m-34w__nmme0mean                      0
contest-wind-h10-14d__wind-hgt-10               0
nmme-tmp2m-56w__cancm3                          0
nmme-tmp2m-56w__cancm4                          0
nmme-tmp2m-56w__ccsm3                       10280
nmme-tmp2m-56w__ccsm4             

DROP NULL VALUS

In [15]:
train = train.dropna()

In [16]:
print('missing values:', train.isnull().sum())

missing values: index                                       0
lat                                         0
lon                                         0
startdate                                   0
contest-pevpr-sfc-gauss-14d__pevpr          0
nmme0-tmp2m-34w__cancm30                    0
nmme0-tmp2m-34w__cancm40                    0
nmme0-tmp2m-34w__ccsm30                     0
nmme0-tmp2m-34w__ccsm40                     0
nmme0-tmp2m-34w__cfsv20                     0
nmme0-tmp2m-34w__gfdlflora0                 0
nmme0-tmp2m-34w__gfdlflorb0                 0
nmme0-tmp2m-34w__gfdl0                      0
nmme0-tmp2m-34w__nasa0                      0
nmme0-tmp2m-34w__nmme0mean                  0
contest-wind-h10-14d__wind-hgt-10           0
nmme-tmp2m-56w__cancm3                      0
nmme-tmp2m-56w__cancm4                      0
nmme-tmp2m-56w__ccsm3                       0
nmme-tmp2m-56w__ccsm4                       0
nmme-tmp2m-56w__cfsv2                       0
nmme-tmp2m-56w__gf

In [ ]:
 #show some information about the dataset
 print(train.info())
print(train.dtypes)


DROP COLUMNS

In [17]:
column_headers = list(train.columns.values)
print("The Column Header :", column_headers)

The Column Header : ['index', 'lat', 'lon', 'startdate', 'contest-pevpr-sfc-gauss-14d__pevpr', 'nmme0-tmp2m-34w__cancm30', 'nmme0-tmp2m-34w__cancm40', 'nmme0-tmp2m-34w__ccsm30', 'nmme0-tmp2m-34w__ccsm40', 'nmme0-tmp2m-34w__cfsv20', 'nmme0-tmp2m-34w__gfdlflora0', 'nmme0-tmp2m-34w__gfdlflorb0', 'nmme0-tmp2m-34w__gfdl0', 'nmme0-tmp2m-34w__nasa0', 'nmme0-tmp2m-34w__nmme0mean', 'contest-wind-h10-14d__wind-hgt-10', 'nmme-tmp2m-56w__cancm3', 'nmme-tmp2m-56w__cancm4', 'nmme-tmp2m-56w__ccsm3', 'nmme-tmp2m-56w__ccsm4', 'nmme-tmp2m-56w__cfsv2', 'nmme-tmp2m-56w__gfdl', 'nmme-tmp2m-56w__gfdlflora', 'nmme-tmp2m-56w__gfdlflorb', 'nmme-tmp2m-56w__nasa', 'nmme-tmp2m-56w__nmmemean', 'contest-rhum-sig995-14d__rhum', 'nmme-prate-34w__cancm3', 'nmme-prate-34w__cancm4', 'nmme-prate-34w__ccsm3', 'nmme-prate-34w__ccsm4', 'nmme-prate-34w__cfsv2', 'nmme-prate-34w__gfdl', 'nmme-prate-34w__gfdlflora', 'nmme-prate-34w__gfdlflorb', 'nmme-prate-34w__nasa', 'nmme-prate-34w__nmmemean', 'contest-wind-h100-14d__wind-hgt

In [19]:

df = train.drop(['wind-vwnd-925-2010-5', 'wind-vwnd-925-2010-6', 'wind-vwnd-925-2010-7', 'wind-vwnd-925-2010-8', 'wind-vwnd-925-2010-9', 'wind-vwnd-925-2010-10', 'wind-vwnd-925-2010-11', 'wind-vwnd-925-2010-12', 'wind-vwnd-925-2010-13', 'wind-vwnd-925-2010-14', 'wind-vwnd-925-2010-15', 'wind-vwnd-925-2010-16', 'wind-vwnd-925-2010-17', 'wind-vwnd-925-2010-18', 'wind-vwnd-925-2010-19', 'wind-vwnd-925-2010-20','nmme-prate-56w__nmmemean', 'contest-pres-sfc-gauss-14d__pres', 'contest-wind-uwnd-250-14d__wind-uwnd-250', 'nmme-tmp2m-34w__cancm3', 'nmme-tmp2m-34w__cancm4', 'nmme-tmp2m-34w__ccsm3', 'nmme-tmp2m-34w__ccsm4', 'nmme-tmp2m-34w__cfsv2', 'nmme-tmp2m-34w__gfdl', 'nmme-tmp2m-34w__gfdlflora', 'nmme-tmp2m-34w__gfdlflorb', 'nmme-tmp2m-34w__nasa', 'nmme-tmp2m-34w__nmmemean', 'contest-prwtr-eatm-14d__prwtr', 'contest-wind-vwnd-250-14d__wind-vwnd-250', 'contest-precip-14d__precip', 'contest-wind-h850-14d__wind-hgt-850', 'contest-wind-uwnd-925-14d__wind-uwnd-925', 'contest-wind-h500-14d__wind-hgt-500', 'cancm30', 'cancm40', 'ccsm30', 'ccsm40', 'cfsv20', 'gfdlflora0', 'gfdlflorb0', 'gfdl0', 'nasa0', 'nmme0mean', 'climateregions__climateregion', 'elevation__elevation', 'wind-vwnd-250-2010-1', 'wind-vwnd-250-2010-2', 'wind-vwnd-250-2010-3', 'wind-vwnd-250-2010-4', 'wind-vwnd-250-2010-5', 'wind-vwnd-250-2010-6', 'wind-vwnd-250-2010-7', 'wind-vwnd-250-2010-8', 'wind-vwnd-250-2010-9', 'wind-vwnd-250-2010-10', 'wind-vwnd-250-2010-11', 'wind-vwnd-250-2010-12', 'wind-vwnd-250-2010-13', 'wind-vwnd-250-2010-14', 'wind-vwnd-250-2010-15', 'wind-vwnd-250-2010-16', 'wind-vwnd-250-2010-17','climateregions__climateregion','contest-pevpr-sfc-gauss-14d__pevpr'], axis=1)

In [20]:
df = df.drop(['startdate'], axis=1)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 352604 entries, 0 to 375733
Columns: 180 entries, index to wind-vwnd-925-2010-4
dtypes: float32(176), int32(1), int8(3)
memory usage: 241.8 MB


MODEL BUILDING

split data into train and test

In [ ]:
x_train = df.drop(['contest-tmp2m-14d__tmp2m'], axis=1) 
y_train = df['contest-tmp2m-14d__tmp2m']

In [23]:
import sklearn.model_selection
# Split train_data
from sklearn.model_selection import train_test_split

X_Train, X_Val, y_Train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.1, random_state=42)

In [25]:
from sklearn import linear_model
model = linear_model.LinearRegression()

In [26]:
x_train.dtypes

index                                         int32
lat                                         float32
lon                                         float32
nmme0-tmp2m-34w__cancm30                    float32
nmme0-tmp2m-34w__cancm40                    float32
nmme0-tmp2m-34w__ccsm30                     float32
nmme0-tmp2m-34w__ccsm40                     float32
nmme0-tmp2m-34w__cfsv20                     float32
nmme0-tmp2m-34w__gfdlflora0                 float32
nmme0-tmp2m-34w__gfdlflorb0                 float32
nmme0-tmp2m-34w__gfdl0                      float32
nmme0-tmp2m-34w__nasa0                      float32
nmme0-tmp2m-34w__nmme0mean                  float32
contest-wind-h10-14d__wind-hgt-10           float32
nmme-tmp2m-56w__cancm3                      float32
nmme-tmp2m-56w__cancm4                      float32
nmme-tmp2m-56w__ccsm3                       float32
nmme-tmp2m-56w__ccsm4                       float32
nmme-tmp2m-56w__cfsv2                       float32
nmme-tmp2m-5

In [27]:
model.fit(x_train, y_train)

LinearRegression()

In [30]:
from sklearn.metrics import r2_score
y_pred = model.predict(X_Val)
r2_score(y_val, y_pred)

0.9758566091597644

## SUBMISSION

In [32]:
# Create submission DataFrame
submission = pd.DataFrame({"uniqueid": test["uniqueid"] + " x " + test["country"],
                           "bank_account": test.bank_account})
submission = pd.DataFrame({"index": test["index"] ,
                           "contest-tmp2m-14d__tmp2m": test.contest-tmp2m-14d__tmp2m)

SyntaxError: invalid syntax (4036796219.py, line 3)

In [ ]:
# Create submission csv file csv file
submission.to_csv('first_submission.csv', index = False)
files.download('first_submission.csv') 